In [1]:
import pandas as pd

In [4]:
import numpy as np

In [5]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
dataset = '/content/drive/My Drive/HBN/tournament_hints_data.parquet'
business_taxonomy_path_labeled = '/content/drive/My Drive/HBN/business_taxonomy_labeled_mpnet.csv'
naics_taxonomy_summary_path = '/content/drive/My Drive/HBN/naics_summary.csv'
output_path = '/content/drive/My Drive/HBN/'

In [9]:
taxonomy_business = pd.DataFrame(pd.read_csv(business_taxonomy_path_labeled))
taxonomy_business

,label,description,naics_label
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...,Motor Vehicle and Parts Dealers
1,Abortion Clinics,Abortion Clinics provide medical services rela...,Hospitals
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...,Administrative and Support Services
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...,Ambulatory Health Care Services
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...,Plastics and Rubber Products Manufacturing
...,...,...,...
598,Work Clothing & Protection Equipment,Work Clothing & Protection Equipment pertains ...,Apparel Manufacturing
599,Writers & Copywriters,Writers & Copywriters are skilled professional...,Publishing Industries
600,YMCA Camps,YMCA Camps offer structured programs and activ...,Accommodation
601,Yoga Studios,Yoga Studios offer a tranquil space for indivi...,Accommodation


In [10]:
taxonomy_naics = pd.DataFrame(pd.read_csv(naics_taxonomy_summary_path))
taxonomy_naics

,Unnamed: 0,naics_code,naics_label,description,summary
0,0,111,Crop Production,Industries in the Crop Production subsector gr...,The Crop Production subsector comprises establ...
1,1,112,Animal Production and Aquaculture,Industries in the Animal Production and Aquacu...,The Animal Production and Aquaculture subsecto...
2,2,113,Forestry and Logging,Industries in the Forestry and Logging subsect...,The forestry and logging industries harvest ti...
3,3,114,"Fishing, Hunting and Trapping","Industries in the Fishing, Hunting and Trappin...","The Fishing, Hunting and Trapping industry rel..."
4,4,115,Support Activities for Agriculture and Forestry,Industries in the Support Activities for Agric...,The Support Activities for Agriculture and For...
...,...,...,...,...,...
91,91,924,Administration of Environmental Quality Programs,The Administration of Environmental Quality Pr...,The Administration of Environmental Quality Pr...
92,92,925,"Administration of Housing Programs, Urban Plan...","The Administration of Housing Programs, Urban ...","The Administration of Housing Programs, Urban ..."
93,93,926,Administration of Economic Programs,The Administration of Economic Programs subsec...,The Administration of Economic Programs subsec...
94,94,927,Space Research and Technology,The Space Research and Technology subsector co...,The Space Research and Technology subsector pr...


In [13]:
loaded_embeddings_descrip_business = np.load(output_path + 'embeddings_taxonomy_description_business.npy')

In [14]:
loaded_embeddings_descrip_business.shape

(603, 768)

In [16]:
loaded_embeddings_summary_naics = np.load(output_path + 'embeddings_summary_naics.npy')

In [17]:
loaded_embeddings_summary_naics.shape

(96, 768)

In [26]:
loaded_embeddings_description_naics = np.load(output_path + 'embeddings_description_naics.npy')

In [18]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_values = []
label = []

for i in range(0,len(loaded_embeddings_descrip_business)):
  results = cosine_similarity(
      [loaded_embeddings_descrip_business[i]],
      loaded_embeddings_summary_naics[0:]
  )
  similarity_values.append(results.max())
  label.append(taxonomy_business.iloc[results.argmax()]['naics_label'])

taxonomy_business.loc[:, "naics_label_summary"] = label
taxonomy_business

,label,description,naics_label,naics_label_summary
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...,Motor Vehicle and Parts Dealers,Ambulatory Health Care Services
1,Abortion Clinics,Abortion Clinics provide medical services rela...,Hospitals,Scenic and Sightseeing Transportation
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...,Administrative and Support Services,"Religious, Grantmaking, Civic, Professional, a..."
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...,Ambulatory Health Care Services,Hospitals
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...,Plastics and Rubber Products Manufacturing,Primary Metal Manufacturing
...,...,...,...,...
598,Work Clothing & Protection Equipment,Work Clothing & Protection Equipment pertains ...,Apparel Manufacturing,Publishing Industries
599,Writers & Copywriters,Writers & Copywriters are skilled professional...,Publishing Industries,"Amusement, Gambling, and Recreation Industries"
600,YMCA Camps,YMCA Camps offer structured programs and activ...,Accommodation,Administrative and Support Services
601,Yoga Studios,Yoga Studios offer a tranquil space for indivi...,Accommodation,Scenic and Sightseeing Transportation


In [27]:
taxonomy_business.loc[:, "score_naics_label_summary"] = similarity_values

In [30]:
similarity_values = []
label = []

for i in range(0,len(loaded_embeddings_descrip_business)):
  results = cosine_similarity(
      [loaded_embeddings_descrip_business[i]],
      loaded_embeddings_description_naics[0:]
  )
  similarity_values.append(results.max())

taxonomy_business.loc[:, "score_naics_label"] = similarity_values
taxonomy_business

,label,description,naics_label,naics_label_summary,score_naics_label_summary,score_naics_label
0,ATVs Dealers & Services,Businesses categorized under ATVs Dealers & Se...,Motor Vehicle and Parts Dealers,Ambulatory Health Care Services,0.636035,0.652380
1,Abortion Clinics,Abortion Clinics provide medical services rela...,Hospitals,Scenic and Sightseeing Transportation,0.395590,0.402203
2,Accounting & Bookkeeping Services,Accounting & Bookkeeping Services encompass a ...,Administrative and Support Services,"Religious, Grantmaking, Civic, Professional, a...",0.498787,0.503622
3,Acupuncture clinic,An Acupuncture clinic is a healthcare facility...,Ambulatory Health Care Services,Hospitals,0.449499,0.355998
4,Adhesives & Sealants,Adhesives & Sealants encompass a diverse range...,Plastics and Rubber Products Manufacturing,Primary Metal Manufacturing,0.321484,0.392045
...,...,...,...,...,...,...
598,Work Clothing & Protection Equipment,Work Clothing & Protection Equipment pertains ...,Apparel Manufacturing,Publishing Industries,0.410678,0.444697
599,Writers & Copywriters,Writers & Copywriters are skilled professional...,Publishing Industries,"Amusement, Gambling, and Recreation Industries",0.403789,0.421462
600,YMCA Camps,YMCA Camps offer structured programs and activ...,Accommodation,Administrative and Support Services,0.327790,0.374180
601,Yoga Studios,Yoga Studios offer a tranquil space for indivi...,Accommodation,Scenic and Sightseeing Transportation,0.227261,0.251861
